In [3]:
# 📁 프로젝트 구조
#
# dart_webapp/
# ├─ app.py                   # Streamlit 메인 앱 (브라우저 UI)
# ├─ utils_dart.py            # DART 코퍼코드 캐시 & 재무데이터 수집/가공
# ├─ finance_korea.py         # 한국 주식(네이버) 시세/상장주식수 스크레이핑
# ├─ requirements.txt         # 필요 라이브러리
# └─ .env                     # 환경변수 (CRTFC_KEY=...)

# =======================
# requirements.txt
# =======================
# streamlit
# pandas
# numpy
# requests
# python-dotenv
# lxml
# beautifulsoup4
# html5lib
# 
# (선택) plotly    # 대화형 차트 원하면 사용

# =======================
# utils_dart.py
# =======================
import io, zipfile, requests, pandas as pd, numpy as np
import xml.etree.ElementTree as ET
from typing import List, Dict, Tuple

DART_CORP_CODE_URL = "https://opendart.fss.or.kr/api/corpCode.xml"
DART_FIN_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"


def build_corp_code_cache(api_key: str, out_csv: str = "data/corp_codes.csv") -> pd.DataFrame:
    """DART에서 CORPCODE.xml 받아 코퍼코드/상장코드 매핑 캐시 생성"""
    import os
    os.makedirs(os.path.dirname(out_csv), exist_ok=True)

    r = requests.get(DART_CORP_CODE_URL, params={"crtfc_key": api_key}, timeout=60)
    r.raise_for_status()

    with zipfile.ZipFile(io.BytesIO(r.content)) as zf:
        xml_bytes = zf.read("CORPCODE.xml")

    root = ET.fromstring(xml_bytes)
    rows = []
    for node in root.findall("list"):
        rows.append({
            "corp_code": node.findtext("corp_code"),
            "corp_name": node.findtext("corp_name"),
            "stock_code": node.findtext("stock_code"),
        })
    df = pd.DataFrame(rows)
    df.to_csv(out_csv, index=False, encoding="utf-8")
    return df


def load_corp_code_cache(csv_path: str = "data/corp_codes.csv") -> pd.DataFrame:
    return pd.read_csv(csv_path, dtype=str)


def find_corp_by_input(df_code: pd.DataFrame, user_input: str) -> Tuple[str, str, str]:
    """회사명 또는 종목코드 입력을 받아 (corp_code, corp_name, stock_code) 반환.
    - 정확 매칭 우선, 없으면 부분 포함 검색 첫 결과 반환
    - 매칭 실패 시 ValueError
    """
    q = user_input.strip()
    # 종목코드(6자리 숫자) 우선
    cand = df_code[df_code["stock_code"].fillna("") == q]
    if not cand.empty:
        row = cand.iloc[0]
        return row["corp_code"], row["corp_name"], row["stock_code"]
    # 회사명 정확
    cand = df_code[df_code["corp_name"].str.fullmatch(q, case=False, na=False)]
    if not cand.empty:
        row = cand.iloc[0]
        return row["corp_code"], row["corp_name"], row["stock_code"]
    # 회사명 부분 포함
    cand = df_code[df_code["corp_name"].str.contains(q, case=False, na=False)]
    if not cand.empty:
        row = cand.iloc[0]
        return row["corp_code"], row["corp_name"], row["stock_code"]
    raise ValueError("해당하는 회사가 없습니다.")


def _to_number(x):
    import re
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s == "" or s == "-":
        return np.nan
    s = s.replace(",", "").replace("원", "").strip()
    if re.match(r"^\(.*\)$", s):
        s = "-" + s[1:-1]
    s = s.replace("△", "-").replace("−", "-")
    try:
        return float(s)
    except:
        return np.nan


def get_financials_json(api_key: str, corp_code: str, years: List[int], reprt_codes=("11011","11012","11013")) -> pd.DataFrame:
    """DART 단일회사 전체재무제표 API 래퍼. IFRS 연결 기준(CFS 우선) 수집 후 피벗.
    - years: [2021,2022,2023,2024] 등
    - reprt_codes: 사업/반기/분기 보고서 분기코드
    반환: (계정명 x 기간) 테이블의 long 형태
    """
    rows = []
    for y in years:
        for rc in reprt_codes:
            params = {
                "crtfc_key": api_key,
                "corp_code": corp_code,
                "bsns_year": str(y),
                "reprt_code": rc,
                "fs_div": "CFS"  # 연결 우선
            }
            resp = requests.get(DART_FIN_URL, params=params, timeout=60)
            resp.raise_for_status()
            data = resp.json()
            if data.get("status") != "013":  # 013: 정정없음/정상 아님. (실사용시 상태코드 로직 조정 가능)
                for it in data.get("list", []) or []:
                    rows.append({
                        "year": y,
                        "reprt_code": rc,
                        "sj_nm": it.get("sj_nm"),       # 재무제표명
                        "account_nm": it.get("account_nm"),
                        "account_id": it.get("account_id"),
                        "thstrm_amount": _to_number(it.get("thstrm_amount")),
                        "frmtrm_amount": _to_number(it.get("frmtrm_amount")),
                        "bfefrmtrm_amount": _to_number(it.get("bfefrmtrm_amount")),
                        "fs_div": it.get("fs_div"),
                    })
    df = pd.DataFrame(rows)
    return df


def add_indicators(df_long: pd.DataFrame) -> pd.DataFrame:
    """핵심 지표(매출, 영업이익, 당기순이익, 자산, 부채, 자본)만 추출하고 Wide 테이블 구성"""
    if df_long.empty:
        return df_long
    key_map = {
        "매출액": ["매출액", "수익(매출액)", "영업수익"],
        "영업이익": ["영업이익"],
        "당기순이익": ["당기순이익", "지배기업 소유주지분 순이익"],
        "자산": ["자산총계", "자산"],
        "부채": ["부채총계", "부채"],
        "자본": ["자본총계", "자본"],
    }
    out_rows = []
    for (y, rc), g in df_long.groupby(["year","reprt_code"]):
        row = {"year": y, "reprt_code": rc}
        for key, aliases in key_map.items():
            val = np.nan
            for a in aliases:
                hit = g[g["account_nm"]==a]["thstrm_amount"]
                if not hit.empty:
                    val = hit.iloc[0]
                    break
            row[key] = val
        out_rows.append(row)
    wide = pd.DataFrame(out_rows).sort_values(["year","reprt_code"]).reset_index(drop=True)
    # 보조지표
    wide["영업이익률(%)"] = (wide["영업이익"] / wide["매출액"]) * 100.0
    wide["ROE(%)"] = (wide["당기순이익"] / wide["자본"]) * 100.0
    return wide


# =======================
# finance_korea.py
# =======================
import re, time
import requests
from bs4 import BeautifulSoup

NAVER_ITEM_MAIN = "https://finance.naver.com/item/main.nhn?code={code}"


def fetch_price_and_shares(stock_code: str) -> Tuple[float, int]:
    """네이버 금융에서 현재가와 상장주식수 파싱 (간단 스크레이핑)
    반환: (price, shares)
    """
    url = NAVER_ITEM_MAIN.format(code=stock_code)
    r = requests.get(url, headers={"User-Agent":"Mozilla/5.0"}, timeout=30)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "lxml")

    # 현재가
    price_tag = soup.select_one(".no_today .blind")
    price = float(price_tag.text.replace(",","")) if price_tag else float("nan")

    # 상장주식수 (종합정보 테이블 하단)
    shares = None
    for th in soup.select("table.tb_type1 th"):
        if "상장주식수" in th.get_text(strip=True):
            td = th.find_next_sibling("td")
            if td:
                m = re.search(r"([0-9,]+)주", td.get_text(" ", strip=True))
                if m:
                    shares = int(m.group(1).replace(",",""))
                    break
    return price, (shares or 0)


def calc_valuation_ratios(price: float, shares: int, equity: float, net_income: float) -> Dict[str, float]:
    """PBR, PER 계산. EPS/ BPS 기준.
    equity: 자본총계, net_income: 당기순이익 (연)
    price: 현재가(원), shares: 상장주식수(주)
    """
    out = {"BPS": float("nan"), "EPS": float("nan"), "PBR": float("nan"), "PER": float("nan")}
    if shares and shares > 0:
        bps = equity / shares if equity else float("nan")
        eps = net_income / shares if net_income else float("nan")
        out["BPS"] = bps
        out["EPS"] = eps
        if bps and bps != 0:
            out["PBR"] = price / bps
        if eps and eps != 0:
            out["PER"] = price / eps
    return out


# =======================
# app.py
# =======================
import os, streamlit as st, pandas as pd
from dotenv import load_dotenv
from utils_dart import build_corp_code_cache, load_corp_code_cache, find_corp_by_input, get_financials_json, add_indicators
from finance_korea import fetch_price_and_shares, calc_valuation_ratios

st.set_page_config(page_title="기업 재무/밸류에이션 뷰어", layout="wide")
st.title("🏢 기업 재무/밸류에이션 뷰어 (DART + Naver)")

with st.sidebar:
    st.header("설정")
    crtfc_key = st.text_input("DART API Key (CRTFC_KEY)", value=os.getenv("CRTFC_KEY", ""), type="password")
    years = st.multiselect("연도 선택", options=list(range(2015, 2026)), default=[2021,2022,2023,2024])
    if st.button("코퍼코드 캐시 갱신"):
        if not crtfc_key:
            st.error("DART API Key를 입력하세요")
        else:
            df = build_corp_code_cache(crtfc_key)
            st.success(f"코퍼코드 {len(df):,}건 캐시 생성 완료")

load_dotenv(dotenv_path=".env")

st.markdown("회사 **이름** 또는 **종목코드(6자리)** 를 입력하세요. 예: `삼성전자` 또는 `005930`")
user_q = st.text_input("검색어", "000660")

col1, col2 = st.columns([1, 2])
with col1:
    if st.button("조회"):
        st.session_state["do_search"] = True

if st.session_state.get("do_search"):
    try:
        # 코퍼코드 로딩
        try:
            df_code = load_corp_code_cache()
        except Exception:
            if not crtfc_key:
                st.error("코퍼코드 캐시가 없고 API Key도 없습니다. 사이드바에서 Key 입력 후 캐시를 생성하세요.")
                st.stop()
            df_code = build_corp_code_cache(crtfc_key)

        corp_code, corp_name, stock_code = find_corp_by_input(df_code, user_q)
        st.subheader(f"회사: {corp_name} ({stock_code or '비상장'})")

        # 재무 수집
        if not crtfc_key:
            st.error("DART API Key가 필요합니다. 사이드바에 입력하세요.")
            st.stop()
        raw = get_financials_json(crtfc_key, corp_code, years)
        fin = add_indicators(raw)

        if fin.empty:
            st.warning("재무데이터가 비어있습니다. 보고서 공개 여부/시점을 확인하세요.")
        else:
            st.dataframe(fin)
            # 최근 연도(사업보고서 코드 11011) 우선으로 밸류에이션 계산
            latest = fin.sort_values(["year","reprt_code"], ascending=[False, True]).iloc[0]
            equity = latest.get("자본") or 0.0
            net_income = latest.get("당기순이익") or 0.0

            val_box = st.container()
            if stock_code and isinstance(stock_code, str) and len(stock_code) == 6:
                price, shares = fetch_price_and_shares(stock_code)
                ratios = calc_valuation_ratios(price, shares, equity, net_income)
                with val_box:
                    st.markdown("### 밸류에이션")
                    st.write({
                        "현재가": price,
                        "상장주식수": shares,
                        "BPS": ratios["BPS"],
                        "EPS": ratios["EPS"],
                        "PBR": ratios["PBR"],
                        "PER": ratios["PER"],
                    })
            else:
                with val_box:
                    st.info("비상장 또는 종목코드 없음: PER/PBR 계산을 생략합니다.")

            # 차트 (Plotly가 설치되어 있으면 사용)
            try:
                import plotly.express as px
                chart_cols = ["매출액", "영업이익", "당기순이익"]
                fin_plot = fin[fin["reprt_code"]=="11011"][["year"]+chart_cols].dropna(how="all", subset=chart_cols)
                fin_melt = fin_plot.melt(id_vars="year", var_name="계정", value_name="금액")
                fig = px.line(fin_melt, x="year", y="금액", color="계정", markers=True, title="연간 실적 추이(사업보고서)")
                st.plotly_chart(fig, use_container_width=True)
            except Exception as e:
                st.caption(f"차트 모듈 미설치 또는 오류: {e}")

    except Exception as e:
        st.error(f"오류: {e}")


# =======================
# 실행 방법 (터미널)
# =======================
# 1) 가상환경 생성/활성화 (선택)
#    python -m venv .venv
#    .venv\\Scripts\\activate  (Windows)
# 2) 의존성 설치
#    pip install -r requirements.txt
# 3) 환경변수 설정 (.env 파일 생성)
#    CRTFC_KEY=여기에_오픈DART_API키
# 4) 앱 실행
#    streamlit run app.py
# 5) 브라우저에서 http://localhost:8501 접속


ModuleNotFoundError: No module named 'utils_dart'